In [3]:
import sys
import os

# Adiciona o diretório base ao sys.path
project_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # Ajuste conforme a localização do notebook
if project_path not in sys.path:
    sys.path.insert(0, project_path)

from modules.plot_solutions import plot_solutions
from modules.nsga2 import nsga2_func
from modules.nsga3 import nsga3_func
from modules.nsga3_deap import nsga3_deap_func
from modules.nsga3_pymoo import nsga3_pymoo_func

In [ ]:
def func1_2d(x):
    return x[0]**2

def func2_2d(x):
    return (x[0] - 2)**2

In [ ]:
bounds_2d = [(-5, 5)]
functions_2d = [func1_2d, func2_2d]
NUM_GEN = 16
POP_SIZE = 52

In [ ]:
import random
import numpy as np

def simple_crossover(parent1, parent2):
    w = random.uniform(0, 1)
    child1 = w*parent1 + (1-w)*parent2
    w = random.uniform(0, 1)
    child2 = w*parent1 + (1-w)*parent2
    return (child1, child2)

def simple_mutation(individual, bounds):
    if random.random() < 0.5: # mutation rate
        for i in range(len(individual)):
            individual[i] += random.uniform(-0.1, 0.1)
            individual[i] = np.clip(individual[i], bounds[i][0], bounds[i][1])
    return individual


In [ ]:
import random
import numpy as np

initial_population = [
            np.array([random.uniform(b[0], b[1]) for b in bounds_2d])
            for _ in range(POP_SIZE)
        ]

In [ ]:
pareto_front_my_nsga2 = nsga2_func(
    POP_SIZE, 
    NUM_GEN, 
    bounds_2d, 
    functions_2d, 
    simple_crossover, 
    simple_mutation,
    initial_population)

plot_solutions(pareto_front_my_nsga2)

In [ ]:
pareto_front_my_nsga3 = nsga3_func(
    POP_SIZE, 
    NUM_GEN, 
    bounds_2d, 
    functions_2d, 
    simple_crossover, 
    simple_mutation,
    initial_population)

plot_solutions(pareto_front_my_nsga3)